In [1]:
# Dependencies and Setup
import gmaps
import pandas as pd
import os
import matplotlib.pyplot as plt
import requests
from config import api_key
import random 
import numpy as np
import requests
import json

# Google developer API key
from googleconfig import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

# File to Load 
cityResult = "Resources/cityWeatherResult.csv"

# Read cities result and put them into Panda dataframe
cityResult_df = pd.read_csv(cityResult)

cityResult_df

,Country,City,Latitude,Longitude,Hemisphere,Processed,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,ag,liberta,17.033333,-61.783333,N,Y,81.09,73,75,5.75
1,am,arevik,40.100556,44.096944,N,Y,30.20,86,100,1.14
2,at,grodig,47.733333,13.033333,N,Y,44.38,57,75,16.11
3,at,koflach,47.066667,15.083333,N,Y,36.79,87,20,1.14
4,at,schladming,47.383333,13.683333,N,Y,33.98,94,100,6.98
...,...,...,...,...,...,...,...,...,...,...
595,vn,bien hoa,10.950000,106.816667,N,Y,77.00,78,40,3.44
596,vn,da nang,16.067778,108.220833,N,Y,68.00,88,40,1.14
597,vn,rach gia,10.016667,105.083333,N,Y,75.20,94,99,5.75
598,za,secunda,-26.550000,29.166667,S,Y,73.00,80,96,3.96


In [2]:
gmaps.configure(api_key=gkey)

# Store 'Latitude' and 'Longitude' into coordinates 
coordinates = cityResult_df[["Latitude", "Longitude"]].astype(float)

# Convert Humdity to float and store
# HINT: be sure to handle NaN values
cityResult_df = cityResult_df.dropna()
humidity_percentage = cityResult_df["Humidity (%)"].astype(float)

# Create a humdity % Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity_percentage, dissipating=False, max_intensity=100,point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Create column to store closing hotel name
cityResult_df["Closest Hotel Name"] = ""

# find the closest Hotel for each ideal city within 5 kilometres radius
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": gkey
}

# Use the search term: "Hotel" and our lat/lng
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in cityResult_df.iterrows():
       #Only process hotel info for Ideal cities where their Temp > 70 and < 80, wind speed < 10, cloudiness = 0 
       if (row['Temperature (F)'] > 70) & (row['Temperature (F)'] < 80) & (row['Wind Speed (mph)'] < 10) & (row['Cloudiness (%)'] == 0):
            # get lat, lng from df
            lat = row["Latitude"]
            lng = row["Longitude"]

            # change location each iteration while leaving original params in place
            params["location"] = f"{lat},{lng}"

            # make request 
            hotel_address = requests.get(base_url, params=params)
            
            #     print the name_address url, avoid doing for public github repos in order to avoid exposing key
            #     print(name_address.url)

            # convert to json
            hotel_address = hotel_address.json()
            # print(json.dumps(name_address, indent=4, sort_keys=True))

            # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
            try:
                cityResult_df.loc[index, "Closest Hotel Name"] = hotel_address["results"][0]["name"]
            except (KeyError, IndexError):
                print("Missing field/result... skipping.")


Missing field/result... skipping.
Missing field/result... skipping.


In [7]:
hotelCityDf = cityResult_df[(cityResult_df['Closest Hotel Name'] !="")]

hotelCityDf
#hotelName = cityResult_df["Closest Hotel Name"].tolist()


,Country,City,Latitude,Longitude,Hemisphere,Processed,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Closest Hotel Name
319,pe,huaral,-11.497500,-77.203333,S,Y,75.20,64,0,8.05,Suite Hotel Las Dalias
402,re,sainte-marie,-20.883333,55.550000,S,Y,72.05,73,0,9.22,Villa Hélèna B&B
518,th,tha mai,12.621369,102.004805,N,Y,77.50,84,0,1.05,Porkaew Resort


In [8]:
# Convert Hotel Names to list
hotelName = hotelCityDf["Closest Hotel Name"].tolist()
# Convert City Names to list
hotelCity = hotelCityDf["City"].tolist()
# Convert Country to list
hotelCountry = hotelCityDf["Country"].tolist()
hotelCoordinates = hotelCityDf[["Latitude", "Longitude"]].astype(float)


# Create hotel symbol layer
#hotelLayer = gmaps.symbol_layer(
 #   hotelCoordinates, fill_color='rgba(0, 150, 0, 0.4)',
  #  stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
   # info_box_content=[f"<b>Hotel name:</b> <br> {hotel} <br> <b>City:</b> <br> {city}  <br> <b>Country:</b> <br> {country} " for hotel,city,country in zip(hotelName,hotelCity,hotelCountry)]
#)

# Create hotel marker layer
hotelMarker = gmaps.marker_layer(hotelCoordinates,info_box_content=[f"<b>Hotel name:</b> <br> {hotel} <br> <b>City:</b> <br> {city}  <br> <b>Country:</b> <br> {country} " for hotel,city,country in zip(hotelName,hotelCity,hotelCountry)])

# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotelMarker)
fig

Figure(layout=FigureLayout(height='420px'))